In [1]:
#IMPORTING LIBRARIES:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
sns.set_style("darkgrid")

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import CountVectorizer

# IF YOU ARE MISSING "WordCloud":
# TRY INSTALLING VIA TERMINAL LIKE THIS: /anaconda3/bin/python -m pip install wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
# IF YOU ARE MISSING "wordninja":
# TRY INSTALLING VIA TERMINAL LIKE THIS: pip install wordninja
import wordninja


%matplotlib inline
import scattertext as st
import re, io
from pprint import pprint
from scipy.stats import rankdata, hmean, norm
import spacy
import os, pkgutil, json, urllib
from urllib.request import urlopen
from IPython.display import IFrame
from IPython.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer
display(HTML("<style>.container { width:98% !important; }</style>"))

In [2]:
#IMPORTING CSV FILES from what was extracteed recently and data retreived from kaggle
depression = pd.read_csv('../data/depression.csv')
suicide_watch = pd.read_csv('../data/suicide_watch.csv')
depression_old = pd.read_csv('../data/OLD_DATA/depression.csv')
suicide_old =pd.read_csv('../data/OLD_DATA/suicide_watch.csv')
new_data=pd.read_csv("../data/Suicide_Detection.csv")

In [3]:
new_data=new_data.drop(columns="Unnamed: 0")
new_data

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide
...,...,...
232069,If you don't like rock then your not going to ...,non-suicide
232070,You how you can tell i have so many friends an...,non-suicide
232071,pee probably tastes like salty tea😏💦‼️ can som...,non-suicide
232072,The usual stuff you find hereI'm not posting t...,suicide


In [4]:
#PEEKING INTO THE DEPRESSION DATA SET
pd.set_option('display.max_columns', 500)
depression.head(-5)

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,top_awarded_type,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,ups,total_awards_received,media_embed,author_flair_template_id,is_original_content,user_reports,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,can_mod_post,score,approved_by,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,content_categories,is_self,mod_note,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,view_count,archived,no_follow,is_crosspostable,pinned,over_18,all_awardings,awarders,media_only,can_gild,spoiler,locked,author_flair_text,treatment_tags,visited,removed_by,num_reports,distinguished,subreddit_id,author_is_blocked,mod_reason_by,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,mod_reports,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,call_to_action,author_cakeday,is_suicide
0,NaN,depression,Welcome to /r/depression's check-in post - a p...,t2_64qjj,False,NaN,0,False,"Regular check-in post, with information about ...",[],r/depression,False,0,NaN,0,NaN,False,t3_xgpxny,False,dark,0.97,NaN,public,34,0,{},NaN,False,[],NaN,False,False,NaN,{},NaN,False,34,NaN,False,False,NaN,1663432672.0,NaN,[],{},NaN,True,NaN,1.663430e+09,text,0,NaN,NaN,text,self.depression,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",NaN,new,NaN,NaN,False,False,False,False,False,[],[],False,False,False,False,NaN,[],False,NaN,NaN,moderator,t5_2qqqf,False,NaN,NaN,NaN,xgpxny,True,NaN,circinia,NaN,122,False,no_ads,False,[],False,NaN,/r/depression/comments/xgpxny/regular_checkin_...,no_ads,True,https://www.reddit.com/r/depression/comments/x...,906655,1.663430e+09,0,NaN,False,NaN,NaN,0
1,NaN,depression,We understand that most people who reply immed...,t2_1t70,False,NaN,3,False,Our most-broken and least-understood rules is ...,[],r/depression,False,0,NaN,0,NaN,False,t3_doqwow,False,dark,1.00,NaN,public,2358,304,{},NaN,False,[],NaN,False,False,NaN,{},NaN,False,2358,NaN,False,True,NaN,False,NaN,[],"{'gid_1': 23, 'gid_2': 3}",NaN,True,NaN,1.572361e+09,text,0,NaN,NaN,text,self.depression,True,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",NaN,confidence,NaN,NaN,True,False,False,False,False,"[{'giver_coin_reward': None, 'subreddit_id': N...",[],False,False,False,False,NaN,[],False,NaN,NaN,moderator,t5_2qqqf,False,NaN,NaN,NaN,doqwow,True,NaN,SQLwitch,NaN,180,True,no_ads,False,[],False,NaN,/r/depression/comments/doqwow/our_mostbroken_a...,no_ads,True,https://www.reddit.com/r/depression/comments/d...,906655,1.572361e+09,2,NaN,False,NaN,NaN,0
2,NaN,depression,"I‘m only turning 22 next year, but it feels sc...",t2_11sxo4,False,NaN,0,False,"I hate getting older, I want to be a kid again...",[],r/depression,False,0,NaN,0,NaN,False,t3_xxjlyw,False,dark,0.95,NaN,public,123,0,{},NaN,False,[],NaN,False,False,NaN,{},NaN,False,123,NaN,False,False,NaN,False,NaN,[],{},NaN,True,NaN,1.665099e+09,text,0,NaN,NaN,text,self.depression,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",NaN,confidence,NaN,NaN,False,False,False,False,False,[],[],False,False,False,False,NaN,[],False,NaN,NaN,NaN,t5_2qqqf,False,NaN,NaN,NaN,xxjlyw,True,NaN,Myechomyshadowandme,NaN,24,True,no_ads,False,[],False,NaN,/r/depression/comments/xxjlyw/i_hate_getting_o...,no_ads,False,https://www.reddit.com/r/depression/comments/x...,906655,1.665099e+09,0,NaN,False,NaN,NaN,0
3,NaN,depression,"Okay, so. Trigger warnings up the wazoo. \n\nI...",t2_2vma8ndr,False,NaN,0,False

In [5]:
depression[depression["author_fullname"]=="t2_64qjj"]

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,top_awarded_type,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,ups,total_awards_received,media_embed,author_flair_template_id,is_original_content,user_reports,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,can_mod_post,score,approved_by,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,content_categories,is_self,mod_note,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,view_count,archived,no_follow,is_crosspostable,pinned,over_18,all_awardings,awarders,media_only,can_gild,spoiler,locked,author_flair_text,treatment_tags,visited,removed_by,num_reports,distinguished,subreddit_id,author_is_blocked,mod_reason_by,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,mod_reports,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,call_to_action,author_cakeday,is_suicide
0,NaN,depression,Welcome to /r/depression's check-in post - a p...,t2_64qjj,False,NaN,0,False,"Regular check-in post, with information about ...",[],r/depression,False,0,NaN,0,NaN,False,t3_xgpxny,False,dark,0.97,NaN,public,34,0,{},NaN,False,[],NaN,False,False,NaN,{},NaN,False,34,NaN,False,False,NaN,1663432672.0,NaN,[],{},NaN,True,NaN,1.663430e+09,text,0,NaN,NaN,text,self.depression,False,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",NaN,new,NaN,NaN,False,False,False,False,False,[],[],False,False,False,False,NaN,[],False,NaN,NaN,moderator,t5_2qqqf,False,NaN,NaN,NaN,xgpxny,True,NaN,circinia,NaN,122,False,no_ads,False,[],False,NaN,/r/depression/comments/xgpxny/regular_checkin_...,no_ads,True,https://www.reddit.com/r/depression/comments/x...,906655,1.663430e+09,0,NaN,False,NaN,NaN,0


In [6]:
#WE'VE PICKED OUT FIVE COLUMNS THAT COULD HELP IN OUR EDA.
#LOOKING AT THOSE COLUMNS FOR r/depression 
depression[["title", "selftext", "author",  "is_suicide","url"]].head(3)

,title,selftext,author,is_suicide,url
0,"Regular check-in post, with information about ...",Welcome to /r/depression's check-in post - a p...,circinia,0,https://www.reddit.com/r/depression/comments/x...
1,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,SQLwitch,0,https://www.reddit.com/r/depression/comments/d...
2,"I hate getting older, I want to be a kid again...","I‘m only turning 22 next year, but it feels sc...",Myechomyshadowandme,0,https://www.reddit.com/r/depression/comments/x...


In [7]:
#WE'VE PICKED OUT FIVE COLUMNS THAT COULD HELP IN OUR EDA.
#LOOKING AT THOSE COLUMNS FOR r/Suicide_Watch

suicide_watch[["title", "selftext", "author",  "num_comments", "is_suicide","url"]].head(3)

,title,selftext,author,num_comments,is_suicide,url
0,New wiki on how to avoid accidentally encourag...,We've been seeing a worrying increase in pro-s...,SQLwitch,251,1,https://www.reddit.com/r/SuicideWatch/comments...
1,Please remember that NO ACTIVISM of any kind i...,"Activism, i.e. advocating or fundraising for s...",SQLwitch,43,1,https://www.reddit.com/r/SuicideWatch/comments...
2,The worst part is when you reach the stage whe...,NaN,Photoboistie,38,1,https://www.reddit.com/r/SuicideWatch/comments...


In [8]:
depression_old[["title", "selftext", "author",  "is_suicide","url"]].head(3)

,title,selftext,author,is_suicide,url
0,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,SQLwitch,0,https://www.reddit.com/r/depression/comments/d...
1,Regular Check-In Post,Welcome to /r/depression's check-in post - a p...,circinia,0,https://www.reddit.com/r/depression/comments/e...
2,I hate it so much when you try and express you...,I've been feeling really depressed and lonely ...,TheNewKiller69,0,https://www.reddit.com/r/depression/comments/f...


In [9]:
suicide_old[["title", "selftext", "author",  "is_suicide","url"]].head(3)

,title,selftext,author,is_suicide,url
0,New wiki on how to avoid accidentally encourag...,We've been seeing a worrying increase in pro-s...,SQLwitch,1,https://www.reddit.com/r/SuicideWatch/comments...
1,Reminder: Absolutely no activism of any kind i...,"If you want to recognise an occasion, please d...",SQLwitch,1,https://www.reddit.com/r/SuicideWatch/comments...
2,To every single poster here i wanne say one thing,I really fucking feel you,NussNougatCroissant,1,https://www.reddit.com/r/SuicideWatch/comments...


In [10]:
#COMPARING SHAPES OF all data
print(depression[["title", "selftext", "author",  "num_comments", "is_suicide","url"]].shape)
print(suicide_watch[["title", "selftext", "author",  "num_comments", "is_suicide","url"]].shape)
print(depression_old[["title", "selftext", "author",  "num_comments", "is_suicide","url"]].shape  )
print(suicide_old[["title", "selftext", "author",  "num_comments", "is_suicide","url"]].shape)
print(new_data.shape)

(976, 6)
(964, 6)
(917, 6)
(980, 6)
(232074, 2)


In [11]:
print(new_data['text'][128][:200])
(len(new_data['text'][128][:200]))
print()

i don't understand why i'm not good enough.I know there are a lot of people in this subreddit who are in impossibly hard situations and I feel like my problem is so small in comparison. However, it is



In [12]:
#READING A RANDOM POST FROM r/depression
print(depression["selftext"][118][:200])
len(depression["selftext"][118])

It’s never gotten this bad and I don’t know what to do. At this point I really want to improve my life and mentally I would be able to do it I think but I just feel stuck in my body. I constantly feel


976

In [13]:
print(depression_old["selftext"][128][:200])
len(depression_old["selftext"][128])

Since this year started I feel like all I do is keep screwing up, I thought I could make life better compared to last year, but instead I’ve f*cked it up more and want nothing but either the end or to


356

In [14]:
print(suicide_watch["selftext"][118])
len(suicide_watch["selftext"][118])

It always comes back here  , I did my fair share of giving I think  , I think this is it


88

In [15]:
print(suicide_old["selftext"][118])
len(suicide_old["selftext"][118])

I'm an 18 year old with severe depression, anxiety, ADHD, borderline personality disorder and DID at a point in my life. But I'm slowly recovering.

I have always felt like I never belonged to this world, because I am "too imaginative" and "too kind". I behave "not like how people should". I'm mostly well-liked, but I know they think I'm a weirdo. Everyone does.

It's final exam time for us, and I have to secure good marks, otherwise I won't be able to enter any good streams in a good college. It would be the end of my future. My family is in a financial crunch, but they give up everything for me. I am dead-set on helping my friends get through their depression and suicidal tendencies, but I'm failing at that. I have had three attempts, but I was saved by a person, who himself was alexithymic (I think) and suicidal. He still is, and he says if he doesn't do well, he will commit suicide.

I don't know what I'm gonna do if I don't perform the way I should. But I certainly know that I wil

1172

In [16]:
#WE'LL CONCAT THE SELECTED PARTS OF BOTH DATASETS BEFORE CLEANING IT
dep_columns = depression[["title", "selftext", "author",  "num_comments", "is_suicide","url"]]
sui_columns = suicide_watch[["title", "selftext", "author",  "num_comments", "is_suicide","url"]]
combined_data_new = pd.concat([dep_columns,sui_columns],axis=0, ignore_index=True)    
combined_data_new.head(2)

,title,selftext,author,num_comments,is_suicide,url
0,"Regular check-in post, with information about ...",Welcome to /r/depression's check-in post - a p...,circinia,122,0,https://www.reddit.com/r/depression/comments/x...
1,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,SQLwitch,180,0,https://www.reddit.com/r/depression/comments/d...


In [17]:
#WE'LL CONCAT THE SELECTED PARTS OF BOTH DATASETS BEFORE CLEANING IT
dep_columns1 = depression_old[["title", "selftext", "author",  "num_comments", "is_suicide","url"]]
sui_columns1 = suicide_old[["title", "selftext", "author",  "num_comments", "is_suicide","url"]]
combined_data_old = pd.concat([dep_columns,sui_columns],axis=0, ignore_index=True)    
combined_data_old.head(2)

,title,selftext,author,num_comments,is_suicide,url
0,"Regular check-in post, with information about ...",Welcome to /r/depression's check-in post - a p...,circinia,122,0,https://www.reddit.com/r/depression/comments/x...
1,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,SQLwitch,180,0,https://www.reddit.com/r/depression/comments/d...


In [18]:
##combining old data and new data
combined_data = pd.concat([combined_data_old,combined_data_new],axis=0, ignore_index=True) 
combined_data[:20]

,title,selftext,author,num_comments,is_suicide,url
0,"Regular check-in post, with information about ...",Welcome to /r/depression's check-in post - a p...,circinia,122,0,https://www.reddit.com/r/depression/comments/x...
1,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,SQLwitch,180,0,https://www.reddit.com/r/depression/comments/d...
2,"I hate getting older, I want to be a kid again...","I‘m only turning 22 next year, but it feels sc...",Myechomyshadowandme,24,0,https://www.reddit.com/r/depression/comments/x...
3,Passive suicidal ideation / not 'wanting to be...,"Okay, so. Trigger warnings up the wazoo. \n\nI...",Send_Cake_Or_Nudes,66,0,https://www.reddit.com/r/depression/comments/x...
4,being poor is ruining my life,"22, disabled w abusive parents. In a 4 Yr 'lon...",robinivy,10,0,https://www.reddit.com/r/depression/comments/x...
5,Can I please have someone to talk to? I'm havi...,Hello kind person. I always have a lot of suic...,SleepyWizard_LUV,35,0,https://www.reddit.com/r/depression/comments/x...
6,Existence was forced upon us all,I have found it increasingly upsetting that I ...,MethodOfYeetus,6,0,https://www.reddit.com/r/depression/comments/x...
7,If you try to hospitalize someone who has no i...,And all they did was vent to you. Not threaten...,ImAScaredRabbit,2,0,https://www.reddit.com/r/depression/comments/x...
8,Turned 29 today.,I feel so alone. I don’t resonate with anyone ...,idontknow_9319,4,0,https://www.reddit.com/r/depression/comments/x...
9,my new therapist said i should be hospitalised 😃🤠,just started with a new therapist as i’ve been...,ifeelnothinganym0re,3,0,https://www.reddit.com/r/depression/comments/x...


In [19]:
#LOOKING AT AUTHORS WITH THE MOST POSTS
author=combined_data["author"].value_counts()
author[author>1]

[deleted]               26
thesweetpotato282       12
Engineer-Still          10
sadninetiesgirl         10
Otherwise-Relation46     8
                        ..
babybluelovesyou         2
International-Oven98     2
h00le                    2
-excuse-me-wtf-          2
CleanPanda4348           2
Name: author, Length: 1761, dtype: int64

#### #ISOLATING AUTHORS WHO HAVE POSTED MORE THAN ONCE and inspecting data

In [20]:
df_author_counts=(pd.DataFrame(combined_data["author"].value_counts()))
df_author_counts.reset_index(level=0, inplace=True)
authors_posting_more_than_once = list(df_author_counts[df_author_counts["author"]>1]["index"])
authors_posting_more_than_once[:40]

['[deleted]',
 'thesweetpotato282',
 'Engineer-Still',
 'sadninetiesgirl',
 'Otherwise-Relation46',
 'Tetreitan',
 'Sea_Detail8594',
 'MsTVs14',
 'Imaginary_Round_9598',
 '2late4menow',
 'fairykitty444',
 'Impossible-Swan-6091',
 'Suicidal-Cunt',
 'Wild-Camera7441',
 'SQLwitch',
 'Safe_Time1327',
 'PresentFormal2485',
 'CloudsofNothingness',
 'Embarrassed_Cup2524',
 'spencercore',
 'm00nfa1ry',
 'toolateforliving18',
 'Cotton_Skies',
 'dancing_dragon_',
 'sadpencil2',
 'very-sadge',
 'Throwawayhoraayy',
 'MinimumSpiritual2557',
 'BeatSpecial7184',
 'Ispeakinhands',
 'cryioboo',
 'jiminsjulia',
 'Lilchubbyspirit',
 'Wishmunk',
 'Survivorcptsd',
 'New-Mood-9805',
 'VegetableVivid2405',
 'Iscocoawesome',
 'EiraLumi',
 'M31Gavin']

In [21]:
pd.set_option("display.max_colwidth", 100)
combined_data[combined_data["author"]=="cryioboo"].head(5)

,title,selftext,author,num_comments,is_suicide,url
648,I’m quitting therapy,It haven’t worked since 13(my mental health about my emotions have been on decadence)and now my ...,cryioboo,0,0,https://www.reddit.com/r/depression/comments/xvs19g/im_quitting_therapy/
678,Is mexico ok? Should i feel ok?,"I cut myself yesterday, called 911 and told them someone was stalking me and i couldn’t handle i...",cryioboo,1,0,https://www.reddit.com/r/depression/comments/xvk9i3/is_mexico_ok_should_i_feel_ok/
1744,Is mexico safe for mental health?,"I cut myself yesterday, called 911 and told them someone was stalking me and i couldn’t handle i...",cryioboo,1,1,https://www.reddit.com/r/SuicideWatch/comments/xvk5pg/is_mexico_safe_for_mental_health/
2588,I’m quitting therapy,It haven’t worked since 13(my mental health about my emotions have been on decadence)and now my ...,cryioboo,0,0,https://www.reddit.com/r/depression/comments/xvs19g/im_quitting_therapy/
2618,Is mexico ok? Should i feel ok?,"I cut myself yesterday, called 911 and told them someone was stalking me and i couldn’t handle i...",cryioboo,1,0,https://www.reddit.com/r/depression/comments/xvk9i3/is_mexico_ok_should_i_feel_ok/


In [22]:
#READING THE POSTS BY AUTHORS WITH THE MOST POSTS
pd.set_option("display.max_colwidth", 1000)

combined_data[["is_suicide","author","title", "selftext"]][combined_data["author"].isin
                                                           (["circinia",'thesweetpotato282', 'Engineer-Still',"Engineer-Still","thesweetpotato282", 
                                                             "Engineer-Still", "sadninetiesgirl", "Otherwise-Relation46","Tetreitan",
                                                             "Sea_Del8594","MsTVs14", "Imaginary_Round_9598",
                                                             "2late4menow", "fairykitty444", "Impossible-Swan-6091","Suicidal-Cunt", 
                                                             "Wild-Camera7441", "SQLwitch", "Safe_Time1327", 'SQLwitch', 
                                                             "MsTVs14"])].sort_values("author")[:10]

,is_suicide,author,title,selftext
1640,1,2late4menow,back to old habits,"i’m back to vomiting my guts out after every meal. i’ll probably get worse like i did before. vomiting till my throat hurts and burns, till i cant talk. i just cant do this. why must my illness control me? what did i do to deserve this? i’n shaking so badly i don’t even know whats wrong with me i just want to end it. j i can barely etype loke this pleass ehelp me"
3580,1,2late4menow,back to old habits,"i’m back to vomiting my guts out after every meal. i’ll probably get worse like i did before. vomiting till my throat hurts and burns, till i cant talk. i just cant do this. why must my illness control me? what did i do to deserve this? i’n shaking so badly i don’t even know whats wrong with me i just want to end it. j i can barely etype loke this pleass ehelp me"
3701,1,2late4menow,agony,i’m suffering so much i can’t do it anymore\n\ni just want the pain to end…
1761,1,2late4menow,agony,i’m suffering so much i can’t do it anymore\n\ni just want the pain to end…
3640,1,2late4menow,it’ll all be over soon,NaN
1700,1,2late4menow,it’ll all be over soon,NaN
1648,1,2late4menow,i want to cry,why am i so unwell why cant i do anything other people can do why do i have to suffer why am i always in pain why does my stomach hurt why do i vomit why do i have no friends why do i want to cry every day why do i hate school why cant i just be like everyone else why do i have to struggle why havent i killed myself yet
3588,1,2late4menow,i want to cry,why am i so unwell why cant i do anything other people can do why do i have to suffer why am i always in pain why does my stomach hurt why do i vomit why do i have no friends why do i want to cry every day why do i hate school why cant i just be like everyone else why do i have to struggle why havent i killed myself yet
3298,1,Engineer-Still,In my final days,Good luck in life for those that keep going
2306,0,Engineer-Still,I can’t stop crying,I’m truly barely holding on


In [23]:
pd.set_option("display.max_colwidth", 1000)
combined_data[["is_suicide","author","title", "selftext"]][combined_data["author"].isin
                                                           (["SQLwitch"])].sort_values("author")[:10]

,is_suicide,author,title,selftext
1,0,SQLwitch,"Our most-broken and least-understood rules is ""helpers may not invite private contact as a first resort"", so we've made a new wiki to explain it","We understand that most people who reply immediately to an OP with an invitation to talk privately mean only to help, but this type of response usually leads to either disappointment or disaster. it usually works out quite differently here than when you say ""PM me anytime"" in a casual social context. \n\nWe have huge admiration and appreciation for the goodwill and good citizenship of so many of you who support others here and flag inappropriate content - even more so because we know that so many of you are struggling yourselves. We're hard at work behind the scenes on more information and resources to make it easier to give and get quality help here - this is just a small start. \n\nOur new wiki page explains in detail why it's much better to respond in public comments, at least until you've gotten to know someone. It will be maintained at /r/depression/wiki/private_contact, and the full text of the current version is below.\n\n*****\n\n###Summary###\n\n**Anyone who, while a..."
976,1,SQLwitch,"New wiki on how to avoid accidentally encouraging suicide, and how to spot covert incitement","We've been seeing a worrying increase in pro-suicide content showing up here and, and also going unreported. This undermines our purpose here, so we wanted to highlight and clarify our guidelines about both direct and indirect incitement of suicide. \n\nWe've created a wiki that covers these issues. We hope this will be helpful to anyone who's wondering whether something's okay here and which responses to report. It explains in detail why *any* validation of suicidal intent, even an ""innocent"" message like ""if you're 100% committed, I'll just wish you peace"" is likely to increase people's pain, and why it's important to report even subtle pro-suicide comments. The full text of the wiki's current version is below, and it is maintained at [/r/SuicideWatch/wiki/incitement](http://www.reddit.com/r/SuicideWatch/wiki/incitement). \n\nWe deeply appreciate everyone who gives responsive, empathetic, non-judgemental support to our OPs, and we particularly thank everyone who's already been..."
977,1,SQLwitch,Please remember that NO ACTIVISM of any kind is ever allowed here. No matter what day it is.,"Activism, i.e. advocating or fundraising for social change or raising awareness of social issues (and suicide is, inescapably, a social issue) is absolutely against the rules here at all times. \n\nPlease understand that we're all for smart, strategic mental-health and suicide-prevention activism. It's essential to fight against stigma, misinformation, and discrimination, and to fight for research, treatment, accommodation, acceptance, and understanding. Most of us, one way or another, are mental-health activists IRL. \n\nBut activism just doesn't work in a dedicated support space that serves a vulnerable population. We used to allow it but the evidence that it was undermining our primary purpose became overwhelming. We do regret the need for this rule, but the need is inescapable. \n\nOur population is all too well aware of the issues and causes that need support and largely not in a position to take action, so besides the fact that activism is often salt in our community's wound..."
1941,0,SQLwitch,"Our most-broken and least-understood rules is ""helpers may not invite private contact as a first resort"", so we've made a new wiki to explain it","We understand that most people who reply immediately to an OP with an invitation to talk privately mean only to help, but this type of response usually leads to either disappointment or disaster. it usually works out quite differently here than when you say ""PM me anytime"" in a casual social context. \n\nWe have huge admiration and appreciation for the goodwill and good citizenship of so many of you who support o

In [24]:
#READING THE POSTS BY AUTHORS WITH DELETED NAMES/ACCOUNTS
pd.set_option("display.max_colwidth", 1000)

combined_data[["is_suicide","author","selftext"]][combined_data["author"].isin(["[deleted]"])].count()#sort_values("author")[:20]

is_suicide    26
author        26
selftext      26
dtype: int64

## FInding Authors who have posted in both

In [25]:
pd.set_option("display.max_colwidth", 100)

#CREATING A DATAFRAME WITH MEAN VALUES OF is_suicide
#A MEAN VALUE THAT IS NOT ONE OR ZERO WOULD INDICATE THAT THEY POST IN BOTH subreddits
more_than_once_mean_df = combined_data[combined_data["author"].isin(authors_posting_more_than_once)].groupby("author").mean()
more_than_once_mean_df.reset_index(level=0, inplace=True)
#CREATING MASK TO ISOLATE REDDITORS WHO HAVE POSTED IN BOTH FORUMS
double_posters_mask_0 = ((more_than_once_mean_df["is_suicide"]) !=0) 
double_posters_mask_1 = ((more_than_once_mean_df["is_suicide"]) !=1.0) 
#TRYING TO FIND OUT HOW MANY DOUBLE POSTERS THERE ARE
double_posters = more_than_once_mean_df[double_posters_mask_0][double_posters_mask_1].sort_values("num_comments", ascending=False)
print(len(double_posters))
#CREATING LIST OF TOP 7 DOUBLE POSTERS BY COMMENTS
top_double_posters_list= list(double_posters["author"][:20])
top_double_posters_list

36


C:\Users\prana\AppData\Local\Temp\ipykernel_7764\3939375953.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  double_posters = more_than_once_mean_df[double_posters_mask_0][double_posters_mask_1].sort_values("num_comments", ascending=False)


['SQLwitch',
 'very-sadge',
 'amustafa_96',
 'gluemeOTL',
 'SacredJujucc',
 'MatthewP0lska',
 '[deleted]',
 'Fair_Owl_2200',
 'ceaseToXist',
 'Bigbrain12341',
 'Wild-Camera7441',
 'BackgroundClick4903',
 'Mean_Lie4091',
 'Maleficent_Sock_8851',
 'in-the_void',
 'bxtx2020',
 'krobus_doobus',
 'TransitionShot8282',
 'Engineer-Still',
 'Few-Salamander-4483']

##### looking at who have posted in both subreddit and analyzing their text

In [26]:

pd.set_option("display.max_colwidth", 1000)
combined_data[["is_suicide","author","title", "selftext","url"]][combined_data["author"].isin(top_double_posters_list)].sort_values("author")[:20]
    

,is_suicide,author,title,selftext,url
3008,1,BackgroundClick4903,"Close to 50, don't want to start over","After decades of marriage she cheated, abused me, and left me financially wrecked. I have lost every career I have started, have been depressed since I was conscious, and can't stand the thought of continuing like this. For some reason I get out of bed, at some point throughout the day swallow whatever pills I'm supposed to take and wait to feel better. It's not happening. My therapist is great. I have friends. They're great. They don't have time for a daily trainwreck though. I sometimes can't get off the floor. I can't work. I am useless to anyone. Like my ex said when we were fighting, ""you will die alone."" I'm an only child, she meant if I lost her I would have nothing at the end. It just doesn't even matter, her words are hollow. I don't need anyone around, I just need to make this pain stop.",https://www.reddit.com/r/SuicideWatch/comments/xxotgr/close_to_50_dont_want_to_start_over/
2127,0,BackgroundClick4903,Every minute is different,"I am coping with a lot of trauma and depression right now, and it's a roller coaster. Some moments I can't get out of bed, others I am pacing and muttering hateful things to myself, then next moment I'm smoking or shoving snack food in my face. A friend made me laugh the other night, but as soon as we hung up I started crying again. I have trouble eating normally or bathing. Today is a little better than yesterday, I just remembered I still need to take my meds. Fuck depression.",https://www.reddit.com/r/depression/comments/xx889d/every_minute_is_different/
187,0,BackgroundClick4903,Every minute is different,"I am coping with a lot of trauma and depression right now, and it's a roller coaster. Some moments I can't get out of bed, others I am pacing and muttering hateful things to myself, then next moment I'm smoking or shoving snack food in my face. A friend made me laugh the other night, but as soon as we hung up I started crying again. I have trouble eating normally or bathing. Today is a little better than yesterday, I just remembered I still need to take my meds. Fuck depression.",https://www.reddit.com/r/depression/comments/xx889d/every_minute_is_different/
1068,1,BackgroundClick4903,"Close to 50, don't want to start over","After decades of marriage she cheated, abused me, and left me financially wrecked. I have lost every career I have started, have been depressed since I was conscious, and can't stand the thought of continuing like this. For some reason I get out of bed, at some point throughout the day swallow whatever pills I'm supposed to take and wait to feel better. It's not happening. My therapist is great. I have friends. They're great. They don't have time for a daily trainwreck though. I sometimes can't get off the floor. I can't work. I am useless to anyone. Like my ex said when we were fighting, ""you will die alone."" I'm an only child, she meant if I lost her I would have nothing at the end. It just doesn't even matter, her words are hollow. I don't need anyone around, I just need to make this pain stop.",https://www.reddit.com/r/SuicideWatch/comments/xxotgr/close_to_50_dont_want_to_start_over/
2070,0,Bigbrain12341,i failed,i failed the the first quarter of freshman year of highschool. i dont know how to feel. i dont know anymore. i feel like a total dumbass and failure.,https://www.reddit.com/r/depression/comments/xxih6r/i_failed/
130,0,Bigbrain12341,i failed,i failed the the first quarter of freshman year of highschool. i dont know how to feel. i dont know anymore. i feel like a total dumbass and failure.,https://www.reddit.com/r/depression/comments/xxih6r/i_failed/
3312,1,Bigbrain12341,my friends wonr know i killed myself,yay,https://www.reddit.com/r/SuicideWatch/comments/xwujp4/my_friends_wonr_know_i_killed_myself/
1372,1,Bigbrain12341,my friends wonr know i killed myself,yay,https://www.reddit.com/r/SuicideWatch/comments/xwujp4/my_friends_wonr_know_i_killed_my

#### Reviewing Significant Authors

####### SQLWitch and circinia seems to be a moderating presence in both subreddits. Appearing a couple of times to remind redditors about rules like non-activism and not posting pro-suicide posts. SQLWitch's posts hints at the culture both communities, which are similar help-seeking, problem-airing forums. 

###### Aside from SQLWitch and circinia, there are other users who have posted on both forums so we will keep their texts but will remove circinia and SQLWitch's texts 

In [27]:

SQLwitch_index= combined_data[combined_data.author.str.contains("SQLwitch")].index
circinia_index= combined_data[combined_data.author.str.contains("circinia")].index
combined_data.drop(index=SQLwitch_index, inplace=True)
combined_data.drop(index=circinia_index, inplace=True)


In [28]:
## Verifying Data
pd.set_option("display.max_colwidth", 1000)
combined_data[["is_suicide","author","title", "selftext"]][combined_data["author"].isin
                                                           (["SQLwitch",'circinia'])].sort_values("author")[:10]

,is_suicide,author,title,selftext


In [29]:
combined_data

,title,selftext,author,num_comments,is_suicide,url
2,"I hate getting older, I want to be a kid again and finally have the childhood I needed.","I‘m only turning 22 next year, but it feels scary and old. I wish I could go back in time and have different people around me, loving and caring people so I can feel emotionally safe and not have mental illness ruin my life from the age of 12.",Myechomyshadowandme,24,0,https://www.reddit.com/r/depression/comments/xxjlyw/i_hate_getting_older_i_want_to_be_a_kid_again_and/
3,Passive suicidal ideation / not 'wanting to be alive'?,"Okay, so. Trigger warnings up the wazoo. \n\nI was out for drinks last week, got fairly far down the rabbit hole and very, very distinctly remember telling a near total stranger that I didn't really want to be alive. It's like I spilled a secret to myself more than them and it's kind of stuck with me ever since. \n\nIt's true. And worse, it's happened before and I forgot - or didn't quite have the words to express it or clock what was going on with me. \n\nI don't want to die. I don't even have thoughts about hurting myself. It crystallised when I was watching a TikTok where someone asked a question about how you'd feel if a truck was speeding toward you. If I was being honest with myself, it was a sigh of relief. \n\nMy life, by all accounts, is great. I have a loving wife, we've just bought a house, I'm in the career I've always wanted to be, I'm lucky enough to have a lot of people I consider close friends. I've struggled with depression and extremely low self-esteem for my ent...",Send_Cake_Or_Nudes,66,0,https://www.reddit.com/r/depression/comments/xx8t8o/passive_suicidal_ideation_not_wanting_to_be_alive/
4,being poor is ruining my life,"22, disabled w abusive parents. In a 4 Yr 'long distance' relationship with quite literally the sweetest person I could've ever met. The easiest, most healthy relationship. My literal life partner who I wish I could give everything to because he deserves it. Anyway. Im so broke with no savings, trying to get away from abusive parents. And when I say broke I mean broke. Being disabled is hard when you're young, and worse when you're broke and without many job options that won't put you in a wheelchair by 35. I can't run, I can't lift, I can barely walk that far anymore and have 677 mental problems including depression anxiety Insomnia constant fights with eating disorders. I wish life would start for me",robinivy,10,0,https://www.reddit.com/r/depression/comments/xxlrw5/being_poor_is_ruining_my_life/
5,Can I please have someone to talk to? I'm having suicidal ideations and am breaking apart rn.,Hello kind person. I always have a lot of suicidal thoughts. I'm barely keeping alive. But I really need someone to talk to. Anyone out there? I'm under a lot of mental and physical turmoil rn.,SleepyWizard_LUV,35,0,https://www.reddit.com/r/depression/comments/xxaajk/can_i_please_have_someone_to_talk_to_im_having/
6,Existence was forced upon us all,"I have found it increasingly upsetting that I had no say in the terms of my existence. It's crazy that is human we are expected to live life and not only be content but ne great full. I think it would be more concerning to have someone living a miserable life and seeming happy as a clam rather than want to end their life or at least do everything to get out of the situation. Why do psychiatrists act like depression and SI is so unusual and scary? It what millions of people deal with every day, and yet it's treated as something that should be worked on constantly. I'm not going to waste my energy trying to ""fix"" my depressed and ultimately not focusing on my actual life. I bet if any other species of animal reached the intelligence of a human many of them would be suicidal as well. This condition is a product of evolution and I really don't see why people should try so hard to fix it.",MethodOfYeetus,6,0,https://www.reddit.com/r/depression/comments/xxmz4i/existence_was_forced_upon_us_all/
..

In [30]:
combined_data["class"]=combined_data["is_suicide"]
combined_data["label"]=combined_data["is_suicide"]
combined_data["text"]=combined_data["selftext"]

In [31]:
combined_data=combined_data[["text","label","class"]]

In [32]:
combined_data.loc[combined_data["class"] == 1, "class"] = "suicide"
combined_data.loc[combined_data["class"] == 0, "class"] = "depression"

C:\Users\prana\AppData\Local\Temp\ipykernel_7764\174746005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data.loc[combined_data["class"] == 1, "class"] = "suicide"


In [68]:
new_data_suicide=new_data[new_data["class"]=="suicide"]
new_data_non_suicide=new_data[new_data["class"]=="non-suicide"]

In [69]:
new_data_non_suicide["class"]="casual-talk"

C:\Users\prana\AppData\Local\Temp\ipykernel_7764\3285184598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data_non_suicide["class"]="casual-talk"


In [71]:
new_data_suicide["label"]=new_data_suicide["class"]
new_data_suicide.loc[new_data_suicide["class"] == "suicide", "label"] = "1"
# new_data_suicide.head(-5)

C:\Users\prana\AppData\Local\Temp\ipykernel_7764\2206895460.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data_suicide["label"]=new_data_suicide["class"]


#### Combining and merging data

In [53]:
combined_data = pd.concat([combined_data,new_data_suicide],axis=0, ignore_index=True) 

In [ ]:
combined_data= pd.concat([combined_data,new_data_non_suicide],axis=0, ignore_index=True) 


In [54]:
combined_data.head(-5)

,text,label,class
0,"I‘m only turning 22 next year, but it feels scary and old. I wish I could go back in time and have different people around me, loving and caring people so I can feel emotionally safe and not have mental illness ruin my life from the age of 12.",0,depression
1,"Okay, so. Trigger warnings up the wazoo. \n\nI was out for drinks last week, got fairly far down the rabbit hole and very, very distinctly remember telling a near total stranger that I didn't really want to be alive. It's like I spilled a secret to myself more than them and it's kind of stuck with me ever since. \n\nIt's true. And worse, it's happened before and I forgot - or didn't quite have the words to express it or clock what was going on with me. \n\nI don't want to die. I don't even have thoughts about hurting myself. It crystallised when I was watching a TikTok where someone asked a question about how you'd feel if a truck was speeding toward you. If I was being honest with myself, it was a sigh of relief. \n\nMy life, by all accounts, is great. I have a loving wife, we've just bought a house, I'm in the career I've always wanted to be, I'm lucky enough to have a lot of people I consider close friends. I've struggled with depression and extremely low self-esteem for my ent...",0,depression
2,"22, disabled w abusive parents. In a 4 Yr 'long distance' relationship with quite literally the sweetest person I could've ever met. The easiest, most healthy relationship. My literal life partner who I wish I could give everything to because he deserves it. Anyway. Im so broke with no savings, trying to get away from abusive parents. And when I say broke I mean broke. Being disabled is hard when you're young, and worse when you're broke and without many job options that won't put you in a wheelchair by 35. I can't run, I can't lift, I can barely walk that far anymore and have 677 mental problems including depression anxiety Insomnia constant fights with eating disorders. I wish life would start for me",0,depression
3,Hello kind person. I always have a lot of suicidal thoughts. I'm barely keeping alive. But I really need someone to talk to. Anyone out there? I'm under a lot of mental and physical turmoil rn.,0,depression
4,"I have found it increasingly upsetting that I had no say in the terms of my existence. It's crazy that is human we are expected to live life and not only be content but ne great full. I think it would be more concerning to have someone living a miserable life and seeming happy as a clam rather than want to end their life or at least do everything to get out of the situation. Why do psychiatrists act like depression and SI is so unusual and scary? It what millions of people deal with every day, and yet it's treated as something that should be worked on constantly. I'm not going to waste my energy trying to ""fix"" my depressed and ultimately not focusing on my actual life. I bet if any other species of animal reached the intelligence of a human many of them would be suicidal as well. This condition is a product of evolution and I really don't see why people should try so hard to fix it.",0,depression
...,...,...,...
119899,"I'm going to do it within the next few days.All my life I have suffered. All they do is drug me up and send me off. Police found me on a bridge last night, sent me off to a hospital, ""This will be the start of a new chapter, it'll get better"" "" Help is on the way"" The doctors like to say. \n\nNo. I'm done with this world and its lies. And you know what? I'm glad.\n\nI had an internal discussion within the past half hour and I'm certain this is what has to happen. Keeping me alive will make me worse off than dead. This pain is chronic. And it will only get worse, as it has done. No more. I want to be at peace. Shame I was dealt a shitty hand. But so some people's lives go.\n\nI'm just posting here because it wouldn't be bad if for once I could find someone who would really, really understand what I was going thro

### Combining casual conversation, shower thoughts

In [72]:
shower_thoughts=pd.read_csv(r"D:\Pranav_Files\Northwestern University\Classes\MSDS_498_Capstone_Project\Data and Git\Capstone_Project\data\shower_thoughts.csv")
casual_convo= pd.read_csv(r"D:\Pranav_Files\Northwestern University\Classes\MSDS_498_Capstone_Project\Data and Git\Capstone_Project\data\casual_convo.csv")

In [73]:
sdf= pd.DataFrame()
sdf["text"]=shower_thoughts["selftext"]
sdf["author"]=shower_thoughts["author"]
sdf["class"]= "casual-talk"

In [74]:
cdf=pd.DataFrame()
cdf["text"]=casual_convo["selftext"]
cdf["author"]=casual_convo["author"]
cdf["class"]= "casual-talk"

In [75]:
# new_data_non_suicide
combined_data1 = pd.concat([sdf,cdf],axis=0, ignore_index=True) 
combined_data= pd.concat([combined_data,combined_data1],axis=0, ignore_index=True)

In [76]:
new_data_non_suicide=new_data_non_suicide[:20000]
# new_data_non_suicide

In [77]:
# combined_data= pd.concat([combined_data1,new_data_non_suicide],axis=0, ignore_index=True)


In [78]:
combined_data["class"].value_counts()

casual-talk    21628
Name: class, dtype: int64

In [ ]:
# combined_data= pd.concat([combined_data,new_data_non_suicide],axis=0, ignore_index=True) 

In [79]:
####BACKING UP combined_data IN A CSV FILE 
combined_data.to_csv('../data/casual_shower.csv', index = False)

In [2]:
combined_data=pd.read_csv('../data/casual_shower.csv')
combined_data

,text,author,class
0,Welcome to r/CasualConversation! Thank you for...,AutoModerator,casual-talk
1,We've set up a Spotify playlist for our Commun...,KaiRegis,casual-talk
2,I got engaged today to my boyfriend. He doesn’...,ShyChili,casual-talk
3,"Potato Broccoli Cheesy Soup from scratch, ya'l...",GoAhead_BakeACake,casual-talk
4,"Without mentioning what covid is, or just how ...",TheNotStinkyOne,casual-talk
...,...,...,...
21623,"Epic penis havers of r/teenagers, what is the ...",NaN,casual-talk
21624,aight teenagers who are socially adept i need ...,NaN,casual-talk
21625,The fact that I wanted to get help for my eati...,NaN,casual-talk
21626,Quick summary of the film: willy wonka and the...,NaN,casual-talk


In [3]:
#CHECKING FOR MISSING DATA
#THERE ARE ONLY NULL VALUES IN OUR selftext COLUMN
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21628 entries, 0 to 21627
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    21628 non-null  object
 1   author  1628 non-null   object
 2   class   21628 non-null  object
dtypes: object(3)
memory usage: 507.0+ KB


In [4]:
print("There are ", combined_data["text"].isnull().sum(), " NAN Values")
## Inspecting data
combined_data[combined_data["text"].isnull()].head(3)

There are  0  NAN Values


,text,author,class


In [5]:
drop_index=combined_data[combined_data["text"].isnull()].index
combined_data.drop(index=drop_index,inplace=True)

In [6]:
## Checking
combined_data[combined_data["text"].isnull()]


,text,author,class


In [7]:
combined_data=combined_data.reset_index(drop=True)

In [64]:
####BAaCKING UP combined_data IN A CSV FILE 
# combined_data.to_csv('../data/new_suicide_data.csv', index = False)

In [2]:
# combined_data=pd.read_csv('../data/new_suicide_data.csv')

### 2.2 Pre-processing

In [11]:

def processing_text(series_to_process):
    new_list = []
    tokenizer = RegexpTokenizer(r'(\w+)')
    lemmatizer = WordNetLemmatizer()
    for i in range(len(series_to_process)):
        #TOKENISED ITEM(LONG STRING) IN A LIST
        dirty_string = (series_to_process)[i].lower()
        words_only = tokenizer.tokenize(dirty_string) #WORDS_ONLY IS A LIST THAT DOESN'T HAVE PUNCTUATION
        #LEMMATISE THE ITEMS IN WORDS_ONLY
        words_only_lem = [lemmatizer.lemmatize(i) for i in words_only]
        #REMOVING STOP WORDS FROM THE LEMMATIZED LIST
        words_without_stop = [i for i in words_only_lem if i not in stopwords.words("english")]
        #RETURN SEPERATED WORDS INTO LONG STRING
        long_string_clean = " ".join(word for word in words_without_stop)
        new_list.append(long_string_clean)
    return new_list

In [10]:
##Cleaning and processing self text and title and creating seperate coumns
##CHECKING TO SEE NEW COLUMNS WERE ADDED
combined_data["text_clean"] = processing_text(combined_data["text"])

In [12]:
####BAaCKING UP combined_data IN A CSV FILE 
combined_data.to_csv('../data/shower_casual_clean.csv', index = False)

In [13]:
pd.set_option("display.max_colwidth", 1000)
combined_data.head(-5)

,text,author,class,text_clean
0,"Welcome to r/CasualConversation! Thank you for joining and coming to our corner of Reddit. \n\n&gt;The friendlier part of Reddit. Have a fun conversation about anything that is on your mind. Ask a question or start a conversation about (almost) anything you desire. Maybe you'll make some friends in the process.\n\nIf you are here, lurking, feel free to create an account and say hi. \n\nHow are you? What brings you here? \n\n&amp;#x200B;\n\nPS, we got rules, please [read 'em](https://www.reddit.com/r/CasualConversation/about/rules)!",AutoModerator,casual-talk,welcome r casualconversation thank joining coming corner reddit gt friendlier part reddit fun conversation anything mind ask question start conversation almost anything desire maybe make friend process lurking feel free create account say hi brings amp x200b p got rule please read em http www reddit com r casualconversation rule
1,"We've set up a Spotify playlist for our Community - send us your favorite song as a comment and we'll add it to the list.\n\nRules:\n\n* Only one song per person\n* After submitting the song can't be changed\n* Songs need to be SFW\n\nHere's the playlist:\n\n[Community Spotify Playlist](https://open.spotify.com/playlist/4RNtBKTVoJeEIv0sSlsR4T?si=2a90125f416248eb)\n\nLet's get the ball rolling, ladies and gentlemen.",KaiRegis,casual-talk,set spotify playlist community send u favorite song comment add list rule one song per person submitting song changed song need sfw playlist community spotify playlist http open spotify com playlist 4rntbktvojeeiv0sslsr4t si 2a90125f416248eb let get ball rolling lady gentleman
2,"I got engaged today to my boyfriend. He doesn’t want to tell anyone yet, so I’m telling a bunch of internet strangers. I’m so happy!!\n\nDid anything exciting happen for you guys today? Or something «regular», whatever (:\n\nEDIT: Wow, I did not expect so many replies! I’ve got to go to bed now, but thank you so much everyone. It’s been fun!",ShyChili,casual-talk,got engaged today boyfriend want tell anyone yet telling bunch internet stranger happy anything exciting happen guy today something regular whatever edit wow expect many reply got go bed thank much everyone fun
3,"Potato Broccoli Cheesy Soup from scratch, ya'll. \nIt was a fantastic dinner. \n\nGranted it was all I managed to do today. \nBut I'm so happy for that dam soup. \nMade me happy having my family enjoy it and knowing I contributed something. \n\nOne day, my anxiety will be better under control (I'm getting help).\n\nBut today I'm celebrating baby steps.\n\nIn the form of soup.\n\n&amp;nbsp;\n\n&amp;nbsp;\n\n&amp;nbsp;\n\nEdit: You guys...gah. All your likes and words of encouragement. I was feeling a bit like a bum that this (however fantastic) soup was all I managed to accomplish today. \n\nBut you all are making me feel like a Rockstar. \n\nI feel like I have dozens of invisible cheerleaders cheering me on. All your words are echoing in mind. Better than the ruckus that is usually going on in there.\n\n🥲 Thank you.",GoAhead_BakeACake,casual-talk,potato broccoli cheesy soup scratch ya wa fantastic dinner granted wa managed today happy dam soup made happy family enjoy knowing contributed something one day anxiety better control getting help today celebrating baby step form soup amp nbsp amp nbsp amp nbsp edit guy gah like word encouragement wa feeling bit like bum however fantastic soup wa managed accomplish today making feel like rockstar feel like dozen invisible cheerleader cheering word echoing mind better ruckus usually going thank
4,"Without mentioning what covid is, or just how it physically affects you, or even trying to describe covid in the most subtle way. \n\n\nI personally would go along the lines of: you’re doing great staying indoors, now keep it that way. \n\nfor loved ones I would say: prepare for a shortage without being the cause to the shortage.\n\nEDIT: Reading all of your responses and I’m sorry for the losses and u

In [19]:
#CHECKING IF OUR FUNCTIONS INTRODUCED MISSING DATA INTO OUR NEW COLS
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21628 entries, 0 to 21627
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            21628 non-null  int64 
 1   text             21628 non-null  object
 2   author           1628 non-null   object
 3   class            21628 non-null  object
 4   text_clean       21628 non-null  object
 5   selftext_length  21628 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1013.9+ KB


In [20]:
#DOUBLE-CHECKING IF OUR FUNCTIONS INTRODUCED MISSING DATA INTO OUR NEW COLS
combined_data.isnull().sum()

index                  0
text                   0
author             20000
class                  0
text_clean             0
selftext_length        0
dtype: int64

### Length of posts
We should dive in to check out what the number of words are in an average post in each subreddit. 

In [21]:
combined_data=combined_data.reset_index()
combined_data.index

RangeIndex(start=0, stop=21628, step=1)

In [17]:
combined_data["selftext_length"]= [len(combined_data["text_clean"][i]) for i in range(len(combined_data))]

In [29]:
combined_data["class"].unique()

array(['casual-talk'], dtype=object)

In [31]:
# ave_length_dep_post = combined_data["selftext_length"][combined_data["label"] ==0].mean()
# ave_length_sui_post = combined_data["selftext_length"][combined_data["label"] ==1].mean()


# print("Average length of a r/depression post: {}".format(ave_length_dep_post))
# print()
# print()
# print("Average length of a r/SuicideWatch post: {}".format(ave_length_sui_post))

In [37]:
####BAaCKING UP combined_data IN A CSV FILE 
combined_data.to_csv('../data/shower_casual_clean.csv', index = False)

#SAVING combined_data
## combined_data.to_csv('../data/data_for_model_new_final.csv', index = False)

## Need to merge both dataset now

In [2]:
import pandas as pd

In [3]:
#SAVING combined_data
df2=pd.read_csv('../data/data_for_model_new_final.csv')
df2=df2.reset_index()
df2.head(2)

,level_0,index,text,label,class,text_clean,selftext_length
0,0,0,"I‘m only turning 22 next year, but it feels sc...",0,depression,turning 22 next year feel scary old wish could...,158
1,1,1,"Okay, so. Trigger warnings up the wazoo. \n\nI...",0,depression,okay trigger warning wazoo wa drink last week ...,1422


In [6]:
shower=pd.read_csv('../data/shower_casual_clean.csv')

In [7]:
shower

,level_0,index,text,author,class,text_clean,selftext_length
0,0,0,Welcome to r/CasualConversation! Thank you for...,AutoModerator,casual-talk,welcome r casualconversation thank joining com...,330
1,1,1,We've set up a Spotify playlist for our Commun...,KaiRegis,casual-talk,set spotify playlist community send u favorite...,277
2,2,2,I got engaged today to my boyfriend. He doesn’...,ShyChili,casual-talk,got engaged today boyfriend want tell anyone y...,210
3,3,3,"Potato Broccoli Cheesy Soup from scratch, ya'l...",GoAhead_BakeACake,casual-talk,potato broccoli cheesy soup scratch ya wa fant...,498
4,4,4,"Without mentioning what covid is, or just how ...",TheNotStinkyOne,casual-talk,without mentioning covid physically affect eve...,385
...,...,...,...,...,...,...,...
21623,21623,21623,"Epic penis havers of r/teenagers, what is the ...",NaN,casual-talk,epic penis havers r teenager coolest thing pen...,87
21624,21624,21624,aight teenagers who are socially adept i need ...,NaN,casual-talk,aight teenager socially adept need help got ba...,369
21625,21625,21625,The fact that I wanted to get help for my eati...,NaN,casual-talk,fact wanted get help eating disorder showed si...,221
21626,21626,21626,Quick summary of the film: willy wonka and the...,NaN,casual-talk,quick summary film willy wonka chocolate facto...,130


In [12]:
shower[["text"]].isin(["Welcome"])

RangeIndex(start=0, stop=21628, step=1)

In [28]:
shower[shower["text"].str.contains("Welcome")]

drop_index= [0,814]
shower=shower.drop(index=drop_index)


In [34]:
df2.head(1)

,level_0,index,text,label,class,text_clean,selftext_length
0,0,0,"I‘m only turning 22 next year, but it feels sc...",0,depression,turning 22 next year feel scary old wish could...,158


In [33]:
shower.head(1)

,level_0,index,text,author,class,text_clean,selftext_length
1,1,1,We've set up a Spotify playlist for our Commun...,KaiRegis,casual-talk,set spotify playlist community send u favorite...,277


In [36]:
df3=pd.DataFrame()
df3["text"]=df2["text"]
df3["class"]=df2["class"]
df3["clean_text"]=df2["text_clean"]

In [39]:
df4=pd.DataFrame()
df4["text"]=shower["text"]
df4["class"]=shower["class"]
df4["clean_text"]=shower["text_clean"]

In [40]:
print(len(df3),len(df4))

119757 21626


In [41]:
final_data= pd.concat([df3,df4],axis=0, ignore_index=True) 

In [48]:
len(final_data)

141385

In [49]:
final_data.to_csv('../data/final_data_casual_combined_cleaned.csv', index = False)